# Array Snail Traversal (TypeScript)

## 概要

1次元配列を「Snail Traversal（蛇行）」パターンで2次元配列に変換する `snail` メソッドを `Array.prototype` に拡張します。

- 入力: `rowsCount` (行数), `colsCount` (列数)
- 制約: `rowsCount * colsCount === nums.length` でなければ空配列 `[]` を返す
- アルゴリズム: 数学的インデックス計算によるO(n)変換


In [ ]:
// 型定義の拡張
declare global {
    interface Array<T> {
        snail(rowsCount: number, colsCount: number): T[][];
    }
}

// 実装
Array.prototype.snail = function <T>(rowsCount: number, colsCount: number): T[][] {
    // 入力バリデーション
    if (rowsCount * colsCount !== this.length) {
        return [];
    }

    // 結果配列の初期化
    const result: T[][] = [];
    for (let i = 0; i < rowsCount; i++) {
        result[i] = [];
    }

    // Snail traversal pattern
    for (let i = 0; i < this.length; i++) {
        // 列番号: インデックスを行数で割った商
        const col = Math.floor(i / rowsCount);
        
        // 列内での位置: インデックスを行数で割った余り
        const positionInCol = i % rowsCount;
        
        // 行番号の決定ルール:
        // 偶数列(0, 2, 4...): 上から下へ (row = positionInCol)
        // 奇数列(1, 3, 5...): 下から上へ (row = rowsCount - 1 - positionInCol)
        const row = col % 2 === 0 ? positionInCol : rowsCount - 1 - positionInCol;
        
        result[row][col] = this[i];
    }

    return result;
};

## 動作確認

### ケース1: 基本的な例
```typescript
const nums = [19, 10, 3, 7, 9, 8, 5, 2, 1, 17, 16, 14, 12, 18, 6, 13, 11, 20, 4, 15];
const result = nums.snail(5, 4);
console.log(result);
```
期待される出力:
```
[
 [19,17,16,15],
 [10,1,14,4],
 [3,2,12,20],
 [7,5,18,11],
 [9,8,6,13]
]
```

### ケース2: サイズ不一致
```typescript
[1,2,3].snail(1, 2); // -> []
```

## 実装の詳細解説

1. **型拡張**: `declare global` ブロック内で `Array<T>` インターフェースを拡張し、`snail` メソッドを追加します。
    - `T[][]`を返す: 要素の型 `T` を維持した2次元配列

2. **バリデーション**: `rowsCount * colsCount !== this.length` の場合、直ちに空配列を返します。

3. **インデックス計算**: 各要素 `this[i]` が配置される `(row, col)` を計算します。
    - `col`: `Math.floor(i / rowsCount)`
    - `row`: 列が偶数なら `i % rowsCount`、奇数なら `rowsCount - 1 - (i % rowsCount)`